Can be ignored. There are no meaningful results from BERTopic.

In [ ]:
!pip install bertopic

In [ ]:
!pip install pymorphy2

### Restart Runtime after BERTopic installation!!!

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from bertopic import BERTopic

In [3]:
import os
import pandas as pd
import numpy as np

In [4]:
import nltk
import pymorphy2

## Data Load

In [5]:
Data = '/content/drive/MyDrive/Colab Notebooks/Projects/eva/Data/'

Messages_filename='philologist_samples_final.csv'
Messages_full_filename=os.path.join(Data, Messages_filename)

text='Sample'

In [6]:
df=pd.read_csv(Messages_full_filename, error_bad_lines=False, index_col=False)
data = df[text].tolist()

## Data preparation

In [7]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [8]:
nltk.download('stopwords')
from nltk.corpus import stopwords
ru_stopwords = stopwords.words('russian')
eng_stopwords = stopwords.words('english')
add_stopwords = ['ай',
'ну',
'по',
'это',
'так',
'что',
'вот',
'кого',
'то',
'кто',
'все',
'всё',
'весь',
'каждый',
#
'год',
'делать',
'сидеть',	
'дать',
'работать',	
'жить',
'день',
'просто',
'час',
'идти',	
'пойти',
'сделать',
'написать',
'писать',
'нужно',
'нужный',
'который'
	]
stopwords = eng_stopwords + ru_stopwords + add_stopwords

from collections import Counter
stopwords_dict = Counter(stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
morph = pymorphy2.MorphAnalyzer()
def lemmatize(text):
    words = text.split()
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return  ' '.join(res)

In [10]:
for i, s in enumerate(data):
  s=lemmatize(s)
  data[i] = ' '.join([word for word in tokenizer.tokenize(s) if word not in stopwords_dict])

## Topic Modeling

In [11]:
def TopicModeling(model_name):
  if model_name=='Russian':
    model = BERTopic(language="Russian")
  else:
    model = BERTopic(embedding_model=model_name)
  topics, probabilities = model.fit_transform(data)
  #Intertopic Distance Map
  if len(model.get_topic_freq())>3:
    Chart=model.visualize_topics()
  else:
    Chart='Not enough data for chart'
  #topics in df
  topics_df=model.get_topic_info()
  #add new columns
  for i in range(0,10):
    topics_df['w'+str(i)]=''
  for i in range(0,10):  
    topics_df['p'+str(i)]=0.0000
#add all words and probabilities
  for index,row in topics_df.iterrows():
    topic_details=model.get_topic(row['Topic'])
    for (i,td) in enumerate(topic_details):
      topics_df.at[index,'w'+str(i)]=td[0]
      topics_df.at[index,'p'+str(i)]=td[1]
  return Chart, topics_df

In [12]:
Chart, topics_df = TopicModeling('Russian')

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [13]:
Chart

In [14]:
topics_df

,Topic,Count,Name,w0,w1,w2,w3,w4,w5,w6,w7,w8,w9,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9
0,-1,197,-1_ребёнок_мама_вообще_язык,ребёнок,мама,вообще,язык,очень,жизнь,например,дом,учить,давать,0.016939,0.010503,0.010427,0.009801,0.009794,0.009109,0.008647,0.008509,0.008438,0.008022
1,0,59,0_ребёнок_работа_диплом_мама,ребёнок,работа,диплом,мама,деньга,жена,муж,жизнь,курс,физика,0.016976,0.012633,0.012613,0.012542,0.009639,0.009150,0.009075,0.008858,0.008619,0.008482
2,1,52,1_ребёнок_человек_мама_хотеть,ребёнок,человек,мама,хотеть,очень,сын,жизнь,русский,ехать,родить,0.021465,0.020275,0.019615,0.013782,0.011381,0.009996,0.009665,0.009332,0.008795,0.008164
3,2,40,2_канада_тысяча_деньга_канадский,канада,тысяча,деньга,канадский,доллар,везде,платить,очень,америка,студент,0.021273,0.014560,0.013293,0.013017,0.012727,0.012507,0.012495,0.012165,0.011613,0.009951
4,3,30,3_язык_украинский_русский_английский,язык,украинский,русский,английский,равно,украина,доллар,слово,вообще,страна,0.029323,0.022845,0.021244,0.011762,0.011593,0.011119,0.010442,0.010051,0.009637,0.009262
5,4,28,4_контракт_работа_деньга_бизнес,контракт,работа,деньга,бизнес,платить,например,офис,продавать,долларов,работу,0.016306,0.015869,0.015212,0.013996,0.013285,0.012347,0.011690,0.011676,0.011648,0.010920
6,5,20,5_ваш_вообще_мама_россия,ваш,вообще,мама,россия,должный,московский,любой,очень,америка,чужой,0.019119,0.014907,0.014838,0.013328,0.012293,0.010410,0.010352,0.010150,0.010077,0.009779
7,6,14,6_человек_инженер_вообще_читать,человек,инженер,вообще,читать,стать,очень,диплом,язык,думать,русский,0.015968,0.015785,0.015230,0.014183,0.013841,0.013385,0.013242,0.013000,0.012344,0.012315


In [15]:
Chart, topics_df = TopicModeling('blinoff/roberta-base-russian-v0')

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/blinoff_roberta-base-russian-v0 were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
Chart

In [17]:
topics_df

,Topic,Count,Name,w0,w1,w2,w3,w4,w5,w6,w7,w8,w9,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9
0,-1,147,-1_ребёнок_мама_деньга_хотеть,ребёнок,мама,деньга,хотеть,лет,муж,получить,давать,доллар,мир,0.023114,0.015525,0.012215,0.011650,0.011587,0.010891,0.010224,0.010101,0.009666,0.008943
1,0,115,0_хотеть_английский_диплом_получить,хотеть,английский,диплом,получить,преподавать,ребёнок,давать,платить,учиться,сдать,0.011967,0.011558,0.010534,0.010146,0.009905,0.008931,0.008695,0.008580,0.008526,0.008478
2,1,61,1_ребёнок_муж_мама_сын,ребёнок,муж,мама,сын,замуж,лет,деньга,девка,папа,купить,0.025722,0.023055,0.022634,0.015108,0.014353,0.013827,0.012701,0.011888,0.011192,0.009919
3,2,40,2_работа_хотеть_ставка_лет,работа,хотеть,ставка,лет,давать,получить,взять,ребёнок,продать,зарплата,0.020390,0.013003,0.012271,0.012196,0.012036,0.011602,0.011458,0.010171,0.010088,0.009856
4,3,38,3_ребёнок_ещё_муж_классный,ребёнок,ещё,муж,классный,лет,мама,холодильник,наш,купить,нога,0.022016,0.018512,0.012627,0.010383,0.010267,0.010115,0.010086,0.009807,0.009592,0.008484
5,4,27,4_ребёнок_иметь_хотеть_баба,ребёнок,иметь,хотеть,баба,диплом,американский,человек,страна,строчка,жизнь,0.018044,0.012669,0.011661,0.011441,0.011417,0.010488,0.010078,0.009623,0.009436,0.008278
6,5,12,5_деньга_учиться_муж_пенсия,деньга,учиться,муж,пенсия,наш,зарплата,колледж,лет,тысяча,стать,0.018795,0.017539,0.016811,0.016380,0.013873,0.012488,0.012096,0.011618,0.011259,0.010742


In [18]:
Chart, topics_df = TopicModeling('sberbank-ai/sbert_large_nlu_ru')

In [19]:
Chart

In [20]:
topics_df

,Topic,Count,Name,w0,w1,w2,w3,w4,w5,w6,w7,w8,w9,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9
0,-1,169,-1_ть_работа_деньга_муж,ть,работа,деньга,муж,дом,канада,мир,лет,жизнь,страна,0.016673,0.013418,0.011610,0.009930,0.009647,0.008959,0.008890,0.008846,0.008655,0.008646
1,0,76,0_ребёнок_мама_муж_бабушка,ребёнок,мама,муж,бабушка,сын,дом,деньга,свекровь,жена,квартира,0.028550,0.027361,0.022646,0.012689,0.012631,0.012573,0.012407,0.011814,0.011472,0.011277
2,1,75,1_работа_хотеть_контракт_бизнес,работа,хотеть,контракт,бизнес,зарплата,резюме,платить,получить,нанять,доллар,0.023760,0.013519,0.013210,0.012873,0.012692,0.011960,0.011948,0.011517,0.011396,0.010833
3,2,62,2_язык_учить_английский_курс,язык,учить,английский,курс,диплом,экзамен,преподавать,филфак,читать,учебник,0.016628,0.014603,0.013914,0.012517,0.012376,0.012044,0.011349,0.010158,0.010129,0.009549
4,3,27,3_врач_больница_пациент_медицинский,врач,больница,пациент,медицинский,доллар,медицина,деньга,переводчик,страховка,переводчица,0.032342,0.026322,0.018486,0.016841,0.013272,0.012769,0.011824,0.011519,0.011227,0.011101
5,4,19,4_язык_украинский_русский_украина,язык,украинский,русский,украина,мов,русскоязычный,россия,советский,украинец,учить,0.046675,0.046577,0.033773,0.027204,0.027122,0.021820,0.015891,0.011960,0.011761,0.011540
6,5,12,5_ребёнок_учиться_мир_америка,ребёнок,учиться,мир,америка,людить,россия,русский,физика,лет,родитель,0.037712,0.019265,0.016878,0.015185,0.013637,0.013623,0.012785,0.012590,0.012508,0.011739
